In [ ]:
from statsmodels.tsa.stattools import coint
import pandas as pd
from dotenv import load_dotenv
import os
from statsmodels.tsa.stattools import adfuller
#load paths/credentials
load_dotenv()

def load_data(file_path):
  if file_path.endswith('.parquet'):
    return pd.read_parquet(file_path,engine='fastparquet')
  elif file_path.endswith('.csv'):
    return pd.read_csv(file_path)
def run_adf_test(data):
  data_clean=data.dropna()
  adf_stat,p_value,lags,nobs,crit,icbest = adfuller(data_clean
  )
  #return dictionary of results
  return {
    'adf_stat': adf_stat, 
    'p_value': p_value, 
    'lags': lags, 
    'nobs': nobs, 
    'crit': crit, 
    'icbest': icbest
  }

def get_close_cols(df):
  close_cols = [col for col in df.columns if 'Close' in col]
  df_close = df[['Date'] + close_cols ]
  return df_close
def find_cointegrated_pairs(significance=0.05,alpha=0.05):
  file_path = os.getenv('stock_data_path')
  df = load_data(file_path)
  df=get_close_cols(df)
  #drop na
  Tickers = ['NVDA', 'AMD', 'MSFT', 'GOOGL', 'AAPL', 'V', 'MA', 'CRM', 'ADBE', 'INTC', 'QCOM', 'CSCO', 'ANET', 'ORCL', 'SAP', 'UBER', 'LYFT', 'META', 'SNAP']
  print(df.columns)
  p_vals = [
    run_adf_test(
      df[f"Close__{ticker}"].dropna()
    ) 
    for ticker in Tickers
    ]
  # close prices are standard for determining nonstationarity
  for i,ticker in enumerate(Tickers):
    if p_vals[i]['p_value'] > significance:
      print(f"{ticker} is nonstationary with p-value {p_vals[i]['p_value']}")
    else:
      print(f"{ticker} is stationary with p-value {p_vals[i]['p_value']}")

  # Test all combinations
  # Return p-values and optimal pairs
find_cointegrated_pairs()

Index(['Date', 'Close__AAPL', 'Close__ADBE', 'Close__AMD', 'Close__ANET',
       'Close__CRM', 'Close__CSCO', 'Close__GOOGL', 'Close__INTC',
       'Close__LYFT', 'Close__MA', 'Close__META', 'Close__MSFT', 'Close__NVDA',
       'Close__ORCL', 'Close__QCOM', 'Close__SAP', 'Close__SNAP',
       'Close__UBER', 'Close__V'],
      dtype='object')
NVDA is nonstationary with p-value 0.9932656284933717
AMD is nonstationary with p-value 0.2815797407491478
